# INSTRUCTIONS

Run each cell in order. Put or extract data into relevant folders as defined in section 2. Make remaining folders in drive.  **Cells denoted with * may require extra action.**


```
Folder Structure Suggested for Section 2:
drive/
    train_data/ <- input midis go here
    train_output/
        graphs/ <- train/val accuracy plots go here
        intermed/ <- intermediate weights, preprocessing go here
        stats/ (currently not being used)
        midi/ <- output midis go here
```

# 1 Imports

In [1]:
# Import Data Manip, Debug
import glob
import pickle
import numpy as np
import pandas as pd
import pdb
import matplotlib.pyplot as plt

# Import Music21
!pip install music21
from music21 import converter, instrument, note, chord, stream

# Import Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import CuDNNLSTM
from keras.layers import CuDNNGRU
from keras.layers import Activation
from keras.layers import Bidirectional
from keras.layers import RepeatVector
from keras.layers import Flatten, Conv1D, BatchNormalization, MaxPooling1D
from keras.regularizers import L1L2
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.callbacks import History

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


# 2 Mount and Set Directories*

In [2]:
DATA_FOLDER = 'drive/train_data/folk'
GRAPHS_FOLDER = 'drive/train_output/graphs'
MIDI_OUTPUT_FOLDER = 'drive/train_output/midi'
INTERMED_FOLDER = 'drive/train_output/intermed'
STATS_FOLDER = 'drive/train_output/stats'

# data specific
EDM_CORPUS = '/edm_'
FOLK_CORPUS = '/folk_'
# SET THIS TO ONE OF ABOVE (ACTION)
CORPUS = FOLK_CORPUS

# 3 Set Training Parameters*

In [3]:
# SET PARAMETERS (ACTION)
RESTS = False
# SPECIFY PARAMETERS TO TEST AS LIST
DROPOUTS = [1]
MODEL_SIZES = [512]
EPOCHS = 10

# 4 Preprocess Data (MIDI Data Into Notes Corpus and Duration Corpus)

In [4]:
# Test

notes_corpus = []
durations_corpus = []

for file in glob.glob(DATA_FOLDER + "/*.mid"):
    try:
      print("Extracting MIDI File: ", file)
      midi_stream = converter.parse(file)

      notes = None

      partition = instrument.partitionByInstrument(midi_stream)

      if not RESTS:
        # No rests
        if partition: 
            notes = partition.parts[0].recurse()
        else: 
            notes = midi_stream.flat.notes
      else:
        # With rests
        if partition: 
            notes = partition.parts[0].recurse()
        else: 
            notes = midi_stream.notesAndRests
          
      in_song_notes = []
      in_song_durations = []
      for element in notes:
          in_song_durations.append(element.duration.quarterLength)
          if isinstance(element, note.Note):
              in_song_notes.append(str(element.pitch))
          elif RESTS and isinstance(element, note.Rest):
              in_song_notes.append("R")
          elif isinstance(element, chord.Chord):
              in_song_notes.append(element.root().nameWithOctave)
#               in_song.append('.'.join(str(n) for n in element.normalOrder))
    except:
      pass
    
    notes_corpus.append(in_song_notes)
    durations_corpus.append(in_song_durations)
            
# Write
with open(INTERMED_FOLDER + CORPUS + 'notes_corpus', 'wb+') as filepath:
    pickle.dump(notes_corpus, filepath)
    
with open(INTERMED_FOLDER + CORPUS + 'durations_corpus', 'wb+') as filepath:
    pickle.dump(durations_corpus, filepath)

Extracting MIDI File:  drive/train_data/folk\sessiontune10.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune100.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1000.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1001.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1002.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1003.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1004.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1005.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1006.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1007.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1008.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1009.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune101.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1010.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1011.mid
Extracting MIDI File:  drive/

Extracting MIDI File:  drive/train_data/folk\sessiontune1117.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1118.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1119.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune112.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1120.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1121.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1122.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1123.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1124.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1125.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1126.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1127.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1128.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1129.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune113.mid
Extracting MIDI File:  driv

Extracting MIDI File:  drive/train_data/folk\sessiontune1349.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune135.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1350.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1351.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1352.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1353.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1354.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1355.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1356.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1357.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1358.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1359.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune136.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1360.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1361.mid
Extracting MIDI File:  driv

Extracting MIDI File:  drive/train_data/folk\sessiontune1466.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1467.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1468.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1469.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune147.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1470.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1471.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1472.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1473.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1474.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1475.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1476.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1477.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1478.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1479.mid
Extracting MIDI File:  dri

Extracting MIDI File:  drive/train_data/folk\sessiontune1695.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1696.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1697.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1698.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1699.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune17.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune170.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1700.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1701.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1702.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1703.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1704.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1705.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1706.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune1707.mid
Extracting MIDI File:  drive

...

Extracting MIDI File:  drive/train_data/folk\sessiontune488.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4880.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4881.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4882.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4883.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4884.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4885.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4886.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4887.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4888.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4889.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune489.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4890.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4891.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4892.mid
Extracting MIDI File:  driv

Extracting MIDI File:  drive/train_data/folk\sessiontune4997.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4998.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune4999.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune5.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune50.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune500.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune5000.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune5001.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune5002.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune5003.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune5004.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune5005.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune5006.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune5007.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune5008.mid
Extracting MIDI File:  drive/tr

Extracting MIDI File:  drive/train_data/folk\sessiontune551.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune552.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune553.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune554.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune555.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune556.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune557.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune558.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune559.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune56.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune560.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune561.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune562.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune563.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune564.mid
Extracting MIDI File:  drive/train_data/f

Extracting MIDI File:  drive/train_data/folk\sessiontune673.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune674.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune675.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune676.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune677.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune678.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune679.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune68.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune680.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune681.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune682.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune683.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune684.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune685.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune686.mid
Extracting MIDI File:  drive/train_data/f

Extracting MIDI File:  drive/train_data/folk\sessiontune912.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune913.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune914.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune915.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune916.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune917.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune918.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune919.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune92.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune920.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune921.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune922.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune923.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune924.mid
Extracting MIDI File:  drive/train_data/folk\sessiontune925.mid
Extracting MIDI File:  drive/train_data/f

# 5 Preprocess Corpus Into Train Data

## 5.1 Notes Corpus

In [7]:
# load notes corpus
with open(INTERMED_FOLDER + CORPUS + 'notes_corpus', 'rb') as filepath:
    notes_corpus = pickle.load(filepath)
    
# If doing learning for one song at a time only
flattened_notes_corpus = []
for song_notes in notes_corpus:
    flattened_notes_corpus += song_notes

vocab_size = len(set(flattened_notes_corpus))
print(vocab_size)

# Produce input output sequences
window_size = 30
note_sequence_input = []
next_note_output = []

notes = sorted(set(flattened_notes_corpus))
note2int = dict((note, num) for num, note in enumerate(notes))

for i in range(0, len(notes_corpus)):
    for j in range(0, len(notes_corpus[i]) - window_size):
        current_sequence = [note2int[note] for note in notes_corpus[i][j:window_size+j]]
        next_note = note2int[notes_corpus[i][window_size+j]]
        note_sequence_input.append(current_sequence)
        next_note_output.append(next_note)

43


In [8]:
# check train and label shapes
training_data = np.reshape(note_sequence_input, (len(note_sequence_input), window_size , 1))
training_data = training_data / float(vocab_size)
print('Train shape: ' + str(training_data.shape))
training_label = np_utils.to_categorical(next_note_output)
print('Label shape: ' + str(training_label.shape))

Train shape: (799705, 30, 1)
Label shape: (799705, 43)


## 5.2 Durations Corpus

In [9]:
# load notes corpus
with open(INTERMED_FOLDER + CORPUS + 'durations_corpus', 'rb') as filepath:
    duration_corpus = pickle.load(filepath)

# Learn on one song at a time
flattened_duration_corpus = []
for song_durations in duration_corpus:
    flattened_duration_corpus += song_durations
    
import collections
counter = collections.Counter(flattened_duration_corpus)
print(counter)


duration_vocab_size = len(set(flattened_duration_corpus))
print(duration_vocab_size)

# Produce input output sequences
duration_window_size = 30
duration_sequence_input = []
next_duration_output = []

durations = sorted(set(flattened_duration_corpus))
duration2int = dict((duration, num) for num, duration in enumerate(durations))

print(duration2int)
# Write
with open(INTERMED_FOLDER + "/edm_duration_counter", 'wb+') as filepath:
    pickle.dump(counter, filepath)

for i in range(0, len(duration_corpus)):
    for j in range(0, len(duration_corpus[i]) - duration_window_size):
        current_duration_sequence = [duration2int[note] for note in duration_corpus[i][j:duration_window_size+j]]
        next_duration = duration2int[duration_corpus[i][duration_window_size+j]]
        duration_sequence_input.append(current_duration_sequence)
        next_duration_output.append(next_duration)

Counter({0.5: 648306, 1.0: 97684, Fraction(1, 3): 67067, 0.25: 61598, Fraction(2, 3): 29074, 1.5: 21210, 0.0: 15440, 2.0: 6626, 0.75: 1722, 3.0: 1328, 2.5: 405, Fraction(5, 3): 302, 4.0: 247, 5.0: 160, Fraction(4, 3): 88, 1.25: 59, 3.5: 59, 6.0: 54, 1.75: 47, 4.5: 46, 2.25: 22, 7.0: 10, 6.5: 7, 8.0: 6, Fraction(8, 3): 5, 5.5: 5, 4.25: 4, Fraction(7, 3): 4, Fraction(17, 3): 2, 7.5: 1})
30
{0.0: 0, 0.25: 1, Fraction(1, 3): 2, 0.5: 3, Fraction(2, 3): 4, 0.75: 5, 1.0: 6, 1.25: 7, Fraction(4, 3): 8, 1.5: 9, Fraction(5, 3): 10, 1.75: 11, 2.0: 12, 2.25: 13, Fraction(7, 3): 14, 2.5: 15, Fraction(8, 3): 16, 3.0: 17, 3.5: 18, 4.0: 19, 4.25: 20, 4.5: 21, 5.0: 22, 5.5: 23, Fraction(17, 3): 24, 6.0: 25, 6.5: 26, 7.0: 27, 7.5: 28, 8.0: 29}


In [10]:
# check train and label shapes
duration_training_data = np.reshape(duration_sequence_input, (len(duration_sequence_input), duration_window_size , 1))
duration_training_data = duration_training_data / float(duration_vocab_size)
print('Train shape: ' + str(duration_training_data.shape))
duration_training_label = np_utils.to_categorical(next_duration_output)
print('Label shape: ' + str(duration_training_label.shape))

Train shape: (799705, 30, 1)
Label shape: (799705, 30)


# 6 Train

## 6.1 Helper to Create Model

In [11]:
def create_model(network_input, n_vocab, model_size, dropout):
  model = Sequential()
  model.add(Conv1D(512, 5))
  model.add(MaxPooling1D())
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(LSTM(
      model_size,
      #input_shape=(network_input.shape[1], network_input.shape[2]),
      return_sequences=True,
      dropout=dropout, recurrent_dropout=0.3
  ))
  model.add(CuDNNLSTM(model_size, return_sequences=True))
  model.add(Dropout(dropout))
  model.add(CuDNNLSTM(model_size))
  model.add(Dense(128))
  model.add(Dropout(dropout))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def create_callback_list(model_size, dropout, model_type):
  filepath = INTERMED_FOLDER + '/%skarpathy-model-weights-%s-%s-%s.hdf5' % (CORPUS, model_type, model_size, dropout)
  model_checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
  )
  return [model_checkpoint], filepath

# acc history
def setup_plot(dropout, size):
  plt.title('Model Accuracy vs. Epoc with Dropout=%s Size=%s' % (dropout, size))
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  
def plot_history(history, model_type, dropout, size):
  plt.plot(history.history['acc'], label="%s train accuracy" % model_type)
  plt.plot(history.history['val_acc'], label="%s val accuracy" % model_type)

def save_plot(file_path):
  plt.legend()
  plt.savefig(file_path)
  plt.clf()
  
def predict_duration(model, WEIGHT_PATH):
  # Prediction
  model.load_weights(WEIGHT_PATH)
  starting_sequence = np.random.randint(219, size=duration_window_size)
  pattern_sequence = starting_sequence.tolist()
  prediction_output = []

  duration2note = dict((num, note) for num, note in enumerate(durations))
  print (duration2note)
  
  for i in range(400):
      prediction_input = np.reshape(pattern_sequence, (1, len(pattern_sequence), 1))
      prediction_input = prediction_input / float(duration_vocab_size)

      if i%3 == 0:   
          print('\r', 'Predicting.  Duration: ', i, end='')  
      if i%3 == 1:   
          print('\r', 'Predicting.. Duration: ', i, end='')
      if i%3 == 2:   
          print('\r', 'Predicting...Duration: ', i, end='')
      prediction = model.predict(prediction_input, verbose=0)

#       prediction_values = np.arange(len(prediction[0]))
#       prediction_prob = np.asarray(list(prediction[0])) / float(sum(prediction[0]))

      # Most probable note prediction
      index = np.argmax(prediction)
      note_instance = duration2note[index]
      prediction_output.append(note_instance)
#       index = np.random.choice(prediction_values, 1, p=prediction_prob)
#       note_instance = int2note[int(index[0])]
#       prediction_output.append(note_instance)

      pattern_sequence.append(index)
      pattern_sequence = pattern_sequence[1:len(pattern_sequence)]    

  prediction_output = prediction_output[300:len(prediction_output)]

  # Write
  with open(INTERMED_FOLDER + ("%sduration_prediction_output" % CORPUS), 'wb+') as filepath:
      pickle.dump(prediction_output, filepath)
  
  return prediction_output

def predict_note(model, WEIGHT_PATH):
  model.load_weights(WEIGHT_PATH)
  # Prediction
  starting_sequence = np.random.randint(219, size=window_size)
  pattern_sequence = starting_sequence.tolist()
  prediction_output = []

  int2note = dict((num, note) for num, note in enumerate(notes))
  print (int2note)

  for i in range(400):
      prediction_input = np.reshape(pattern_sequence, (1, len(pattern_sequence), 1))
      prediction_input = prediction_input / float(vocab_size)

      if i%3 == 0:   
          print('\r', 'Predicting.  Note: ', i, end='')  
      if i%3 == 1:   
          print('\r', 'Predicting.. Note: ', i, end='')
      if i%3 == 2:   
          print('\r', 'Predicting...Note: ', i, end='')
      prediction = model.predict(prediction_input, verbose=0)

#       prediction_values = np.arange(len(prediction[0]))
#       prediction_prob = np.asarray(list(prediction[0])) / float(sum(prediction[0]))
#       print(prediction_prob)

      # Most probable note prediction
      index = np.argmax(prediction)
      note_instance = int2note[index]
      prediction_output.append(note_instance)

      # Predict based on prob dist
#       index = np.random.choice(prediction_values, 1, p=prediction_prob)
  #     print(index[0])
  #     print(type(index[0]))
#       note_instance = int2note[int(index[0])]
#       prediction_output.append(note_instance)

      pattern_sequence.append(index)
      pattern_sequence = pattern_sequence[1:len(pattern_sequence)]    

  prediction_output = prediction_output[300:len(prediction_output)]
  # Write
  with open(INTERMED_FOLDER + ("%snotes_prediction_output" % CORPUS), 'wb+') as filepath:
      pickle.dump(prediction_output, filepath)
  return prediction_output

def output_midi(prediction_output, duration_prediction_output, dropout, model_size):
  offset = 0
  output_notes = []
  for pattern, duration in zip(prediction_output, duration_prediction_output):

      if ('.' in pattern) or pattern.isdigit():
          chord_array = pattern.split('.')
          chord_notes = []
          for note_instance in chord_array:
              note_object = note.Note(int(note_instance))
              note_object.duration.quarterLength = duration
              note_object.storedInstrument = instrument.Piano()
              chord_notes.append(note_object)
          chord_object = chord.Chord(chord_notes)
          chord_object.offset = offset
          output_notes.append(chord_object)
      elif 'R' == pattern:
          note_object = note.Rest()
          note_object.duration.quarterLength = duration
          note_object.offset = offset
          output_notes.append(note_object)
      else:
          note_object = note.Note(pattern)
          note_object.duration.quarterLength = duration
          note_object.offset = offset
          note_object.storedInstrument = instrument.Piano()
          output_notes.append(note_object)

      offset += 0.5

  midi_stream = stream.Stream(output_notes)
  midi_stream.write('midi', fp=MIDI_OUTPUT_FOLDER + CORPUS + '%s_%s.mid' % (dropout, model_size))
  print('Wrote midi...')

In [12]:
notes_histories = {}
duration_histories = {}
for dropout in DROPOUTS:
  for model_size in MODEL_SIZES:
    setup_plot(dropout, model_size)
    print('Running duration training on dropout:%s size:%s' % (dropout, model_size))
    duration_callbacks, duration_weight_path = create_callback_list('duration', dropout, model_size)
    duration_model = create_model(duration_training_data, duration_vocab_size, model_size, dropout)
    duration_histories[(dropout, model_size)] = duration_model.fit(duration_training_data, duration_training_label, epochs=EPOCHS, batch_size=1024, callbacks=duration_callbacks, validation_split=0.2)
    plot_history(duration_histories[(dropout, model_size)], 'Durations', dropout, model_size)
    # output intermed duration
    duration_prediction = predict_duration(duration_model, duration_weight_path)
    
    print('\n\nRunning notes training on d:%s s:%s' % (dropout, model_size))
    notes_callbacks, note_weight_path = create_callback_list('notes', dropout, model_size)
    notes_model = create_model(training_data, vocab_size, model_size, dropout)
    notes_histories[(dropout, model_size)] = notes_model.fit(training_data, training_label, epochs=EPOCHS, batch_size=735, callbacks=notes_callbacks, validation_split=0.2)
    plot_history(notes_histories[(dropout, model_size)], 'Notes', dropout, model_size)
    save_plot(GRAPHS_FOLDER + CORPUS + 'dropout=%s_size=%s.jpg' % (dropout, model_size))
    # output intermed notes
    note_prediction = predict_note(notes_model, note_weight_path)
    
    # output final midi
    output_midi(note_prediction, duration_prediction, dropout, model_size)

Running duration training on dropout:1 size:512
Train on 639764 samples, validate on 159941 samples
Epoch 1/10
639764/639764 [==============================] - 181s 282us/step - loss: 0.7720 - acc: 0.7592 - val_loss: 0.9887 - val_acc: 0.7490
Epoch 2/10
639764/639764 [==============================] - 178s 278us/step - loss: 0.6136 - acc: 0.8047 - val_loss: 1.7246 - val_acc: 0.2816
Epoch 3/10
639764/639764 [==============================] - 180s 281us/step - loss: 0.5401 - acc: 0.8262 - val_loss: 1.1759 - val_acc: 0.6237
Epoch 4/10
639764/639764 [==============================] - 181s 282us/step - loss: 0.4974 - acc: 0.8384 - val_loss: 0.7354 - val_acc: 0.7923
Epoch 5/10
639764/639764 [==============================] - 181s 283us/step - loss: 0.4724 - acc: 0.8453 - val_loss: 0.7333 - val_acc: 0.7712
Epoch 6/10
639764/639764 [==============================] - 180s 281us/step - loss: 0.4430 - acc: 0.8540 - val_loss: 0.6821 - val_acc: 0.8015
Epoch 7/10
639764/639764 [======================

<Figure size 432x288 with 0 Axes>